# Initialising Your Agent as an Issuing Authority

### In this notebook we walk through the set of steps you need to take to set up the OM Authority as an issuer of a credential.


## 1. Initialise a controller for Issuer Agent

First pass in the required arguments to the AriesAgentController so it can interact with the om-authority-agent ACA-Py instance. You will be doing this at the start of every notebook.

In [18]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://om-authority-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Get current public decentralised identifier

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

You are encouraged to review the lesson on DID's if you feel you need to.

In [19]:
response = await agent_controller.wallet.get_public_did()
print(response)

{'result': {'did': 'vrzjfm1MEN1g5o6QtHLfv', 'verkey': 'WMgexs48Qs1voyqA215F9cvmdEhVfBexWrqCxPGcxzb', 'posture': 'public'}}


## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verkey for that DID. A representation of the public key associated with this identifier. 

Notice the posture property of the returned object - this DID currently only exists in your agent's wallet.

In [3]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'vrzjfm1MEN1g5o6QtHLfv', 'verkey': 'WMgexs48Qs1voyqA215F9cvmdEhVfBexWrqCxPGcxzb', 'posture': 'wallet_only'}


## 4. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [4]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "vrzjfm1MEN1g5o6QtHLfv": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by vrzjfm1MEN1g5o6QtHLfv to the ledger with role ENDORSER"}}'}
200


## 5. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

### This will not work until you have accepted the TAA

In [5]:
try:
    response = await agent_controller.wallet.assign_public_did(did_object["did"])
except:
    print("This failed because you have not signed the TAA")

This failed because you have not signed the TAA
Error during POST /wallet/did/public: 400, message="Ledger rejected transaction request: client request invalid: InvalidClientTaaAcceptanceError('Txn Author Agreement acceptance is required for ledger with id 1',).", url=URL('http://om-authority-agent:8021/wallet/did/public?did=vrzjfm1MEN1g5o6QtHLfv')


In [6]:
response = await agent_controller.ledger.get_taa()
TAA = response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
print(TAA)

{'digest': '8cee5d7a573e4893b08ff53a0761a22a1607df3b3fcd7e75b98696c92879641f', 'text': '\ufeff# Transaction Author Agreement V2\nhttps://sovrin.org/\n\n\n## Summary:\n\n\nThis summary is provided to help you understand your obligations when writing to\nthe Sovrin Ledger Networks-it does not have any legal effect or replace the full\nlegal text of the agreement provided below it.\n\n\n- This agreement grants you permission to write data to the Sovrin Ledger\n  Networks under certain terms and conditions.\n\n\n- You represent and warrant that the data you are writing does not violate any\n  applicable laws or infringe the rights of any other party.\n\n\n- You understand the data you are writing is public and permanent and there can\n  be no guarantee of erasure. This includes public keys and payment addresses.\n\n\n- If it is determined that the data you wrote violated this agreement, the\n  operators of the network can take steps to block it from public access.\n\n\n- The Sovrin Foundat

In [7]:
response = await agent_controller.ledger.accept_taa(TAA)
## Will return {} if successful
print(response)

{}


## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [8]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'vrzjfm1MEN1g5o6QtHLfv', 'verkey': 'WMgexs48Qs1voyqA215F9cvmdEhVfBexWrqCxPGcxzb', 'posture': 'public'}}


## 7. View your DID on IndyScan

If you navigate to the [IndyScan StagingNet](https://indyscan.io/txs/SOVRIN_STAGINGNET/domain) and search for the DID printed above your should see the transaction you just wrote to this public ledger. 

Pretty cool!

## 8. Get public DID

You can now check your DID is public

In [9]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_nym = response['result']['did']
if response['result']['posture'] == 'public':
    print('Congratulations your OM Authority public DID is:', issuer_nym)
else: 
    print('Uh Oh, you must have done something wrong')

{'result': {'did': 'vrzjfm1MEN1g5o6QtHLfv', 'verkey': 'WMgexs48Qs1voyqA215F9cvmdEhVfBexWrqCxPGcxzb', 'posture': 'public'}}
Congratulations your OM Authority public DID is: vrzjfm1MEN1g5o6QtHLfv


## 9. Fetch verkey for public DID

Additionally, we can verify that this DID does actually resolve to the public key material on the ledger.

In [10]:
issuer_verkey = await agent_controller.ledger.get_did_verkey(issuer_nym)
print(issuer_verkey)

{'verkey': 'WMgexs48Qs1voyqA215F9cvmdEhVfBexWrqCxPGcxzb'}


## 10. Get public DID endpoint

As well as providing a publically accessible endpoint to contact the DID controller through the agent framework. Other agents can use this endpoint to connect with your agent and send it messages. Notice that it is an ngrok endpoint which will expire after 8 hours.

In [11]:
issuer_endpoint = await agent_controller.ledger.get_did_endpoint(issuer_nym)
print(issuer_endpoint)

{'endpoint': 'https://0bd98e2df434.ngrok.io'}


## 12. The Importance of a Public DID

This public identifier you just created for your OM Authority acts as the root of trust for this entity. In more realistic scenario's you would want to have a way to communicate this identifier and it's association to a public entity widely.

Adding it into the .well-known folder for your public website is one way being considered for this, see the specification being developed [here](https://identity.foundation/.well-known/resources/did-configuration/) for more information.

For this tutorial series you will just **copy** this public identifier across to the Data Scientist and Data Owner who will use it to provide assurance that the credentials they are verifying were issued from the OM Authority. Do not worry about this for now, you can always fetch the public DID from the OM Agent when you need it.

## 13. Writing a Schema

A credential schema defines the name, set of attributes and version for a particular credential. Within the Hyperledger stack these are currently required to be defined and stored on the ledger. 

You can write your own by following the comments on the code block below. We have already written the schema used in this course to the ledger for you, but feel free to write a different one.

### OpenMined PKI Course

You will issue this to yourself in the next notebook, learning how to interact with a mobile agent that can be installed from the Google or Apple app store. A record that you could use to prove to others you took this course.

You can view the schema transaction on IndyScan [here](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/188817).

In [12]:
# # Define you schema name - must be unique on the ledger
# schema_name = "OpenMined PKI Course"
# # Can version the schema if you wish to update it
# schema_version = "0.0.1"
# # Define any list of attributes you wish to include in your schema
# attributes = ["name", "date"]

# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# schema_id = response["schema_id"]
# print(schema_id)

### OM Data Scientist

You will issue this to the data-scientist-agent later on in the course.

You can view the schema transaction on IndyScan [here](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/188831).

In [13]:
# # Define you schema name - must be unique on the ledger
# schema_name = "OM Data Scientist"
# # Can version the schema if you wish to update it
# schema_version = "0.0.1"
# # Define any list of attributes you wish to include in your schema
# attributes = ["name", "scope"]

# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# schema_id = response["schema_id"]
# print(schema_id)

### OM Data Owner

You will issue this to the data-owner-agent later in the course.

You can view the schema transaction on IndyScan [here](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/188835).

In [14]:
# # Define you schema name - must be unique on the ledger
# schema_name = "OM Data Owner"
# # Can version the schema if you wish to update it
# schema_version = "0.0.1"
# # Define any list of attributes you wish to include in your schema
# attributes = ["name", "domain"]

# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# schema_id = response["schema_id"]
# print(schema_id)

### Your Custom Schema?

Define and write your own schema to the ledger if you want. Just follow the same pattern in the above cells. 

Be sure to write a definition for this schema and store both the identifiers for later use. You will have to load these and change the attributes as necessary if you wish to issue and present these custom credentials. That is left as an excercise for the interested.

In [15]:
# Your schema ...

## 14. Writing a Credential Definition

A credential definition is a transaction that states on the public ledger that the entity associated with the public DID you created will issue credentials using a certain schema with a specific public key. You do not have to have written the schema to the ledger to be able to write a credential definition pointing to the schema.

This generates a public key for a CL-RSA signature scheme, it will take some time.

In [ ]:
# Note if you defined your own schema you will need to extend this code block to write a cred def for it.
pki_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OpenMined PKI Course:0.0.1"
data_scientist_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1"
data_owner_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1"

response = await agent_controller.definitions.write_cred_def(pki_schema_id)
pki_cred_def_id = response["credential_definition_id"]

response = await agent_controller.definitions.write_cred_def(data_scientist_schema_id)
data_scientist_cred_def_id = response["credential_definition_id"]

response = await agent_controller.definitions.write_cred_def(data_owner_schema_id)
data_owner_cred_def_id = response["credential_definition_id"]

print("PKI CRED DEF :", pki_cred_def_id)
print("Data Scientist Cred Def :", data_scientist_cred_def_id)
print("Data Owner Cred Def :", data_owner_cred_def_id)

## 15. Store The Schema and Credential Definition Identifiers

You will need these later when requesting the OM Authority to issue specific credentials. This is the final step in this notebook in which we have shown a the stages that need to take place for an ACA-Py agent to be initialised with the relevant information written to the public ledger so sign and issue make verifiable attestations to others.

Note that usually you would be storing this information in your application's database. You should only be creating these object's once, but if you stop and start the tutorial you will have to repeat this notebook.

In [17]:
%store pki_schema_id
%store data_scientist_schema_id
%store data_owner_schema_id

Stored 'pki_schema_id' (str)
Stored 'data_scientist_schema_id' (str)
Stored 'data_owner_schema_id' (str)


In [18]:
%store pki_cred_def_id
%store data_scientist_cred_def_id
%store data_owner_cred_def_id

Stored 'pki_cred_def_id' (str)
Stored 'data_scientist_cred_def_id' (str)
Stored 'data_owner_cred_def_id' (str)


## 16. End of Tutorial

This terminates the server that the agent_controller spins up to listen to webhooks. You will need to do it at the end of every noteoobk.

In [19]:
response = await agent_controller.terminate()
print(response)

None


# Continue to 4.

In this next notebook you will learn how to create connections and issue credentials by issuing yourself a OM PKI Course credential to your a mobile agent you can install from the App store.